In [2]:
import numpy as np

# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# 7.4 합성곱/풀링 계층 구현하기

## 7.4.1 4차원 배열

In [ ]:
# 높이 28, 너비 28, 채널 1, 데이터 10개 
x = np.random.rand(10, 1, 28, 28) # 무작위로 데이터 생성
x.shape

In [ ]:
x[0].shape
x[1].shape

In [ ]:
# 첫 번째 데이터의 첫 채널의 공간 데이터에 접근하려면..
x[0, 0] # 또는 x[0][0]
x[0, 0].shape

## 7.4.3 합성곱 계층 구현하기

In [ ]:
im2col(input_data, filter_h, filter_w, stride=1, pad=0)

In [4]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7) # (데이터 수, 채널 수, 높이, 너비)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape) # (9, 75)

x2 = np.random.rand(10, 3, 7, 7) # 데이터 10개
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [5]:
class Convolution:
    # W : Weight => 가중치, 필터
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int((H + 2*self.pad - FH)/self.stride + 1)
        out_w = int((W + 2*self.pad - FW)/self.stride + 1)

        # 입력 데이터를 im2col로 전개
        col = im2col(x, FH, FW, self.stride, self.pad)
        
        # 필터도 reshape을 사용해 2차원 배열로 전개
        col_W = self.W.reshape(FN, -1).T

        # 이렇게 전개한 두 행렬의 곱을 구함
        out = np.dot(col, col_W) + self.b

        # 출력 데이터를 적절한 형상으로 바꿔줌
        # (N, C, OH, OW)
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        return out

## 7.4.4 풀링 계층 구현하기

In [ ]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int((H - self.pool_h)/self.stride + 1)
        out_w = int((W - self.pool_w)/self.stride + 1)

        # 전개 (1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)

        # 최댓값 (2)
        out = np.max(col, axis=1)

        # 성형 (3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        return out

# 7.5 CNN 구현하기

In [7]:
class SimpleConvNet:
    def __init__(self, input_dim=(1,28,28), conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                hidden_size=100, ouput_size=10, weight_init_std=0.01):
                
                filter_num = conv_param['filter_num']
                filter_size = conv_param['filter_size']
                filter_pad = conv_param['pad']
                filter_stride = conv_param['stride']
                input_size = input_dim[1]
                conv_output_size = (input_size - filter_size + 2*filter_pad)/filter_stride + 1
                pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

                self.params = {}
                self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)